# Imports

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
from scipy.optimize import brentq, newton
import matplotlib.pyplot as plt
import pandas as pd

# Notes